## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
weather_data_df = pd.read_csv("WeatherPy_Database.csv")
weather_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lompoc,US,34.6391,-120.4579,73.56,57,0,32.01,clear sky
1,1,Vaini,TO,-21.2000,-175.2000,75.36,78,20,9.22,few clouds
2,2,Mossendjo,CG,-2.9506,12.7261,71.28,97,31,0.22,scattered clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,80.56,76,17,11.23,few clouds
4,4,Deputatskiy,RU,69.3000,139.9000,14.52,97,100,4.18,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the weather_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & \
                                       (weather_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,75.36,78,20,9.22,few clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,80.56,76,17,11.23,few clouds
8,8,Quatre Cocos,MU,-20.2078,57.7625,75.31,86,25,9.33,scattered clouds
12,12,Bengkulu,ID,-3.8004,102.2655,77.14,82,96,4.07,overcast clouds
14,14,Rikitea,PF,-23.1203,-134.9692,76.48,76,87,21.14,overcast clouds
20,20,Lere,NG,10.3870,8.5726,86.07,43,100,5.53,overcast clouds
24,24,Georgetown,MY,5.4112,100.3354,82.33,93,20,2.30,light rain
31,31,Avarua,CK,-21.2078,-159.7750,82.45,83,40,5.75,scattered clouds
33,33,Raga,SS,8.4596,25.6780,80.10,61,100,6.53,overcast clouds
34,34,Mumford,GH,5.2618,-0.7590,82.15,82,100,12.86,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
3,Puerto Ayora,EC,80.56,few clouds,-0.7393,-90.3518,
8,Quatre Cocos,MU,75.31,scattered clouds,-20.2078,57.7625,
12,Bengkulu,ID,77.14,overcast clouds,-3.8004,102.2655,
14,Rikitea,PF,76.48,overcast clouds,-23.1203,-134.9692,
20,Lere,NG,86.07,overcast clouds,10.3870,8.5726,
24,Georgetown,MY,82.33,light rain,5.4112,100.3354,
31,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
33,Raga,SS,80.10,overcast clouds,8.4596,25.6780,
34,Mumford,GH,82.15,overcast clouds,5.2618,-0.7590,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   203
Country                203
Max Temp               203
Current Description    203
Lat                    203
Lng                    203
Hotel Name             203
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/weatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))